#Installazione e Configurazione

In [ ]:
!git clone https://github.com/agwmon/MuDI.git
%cd MuDI

In [ ]:
%pip install diffusers[torch] transformers peft wandb scipy

%pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
!pip install torch torchvision
!pip install matplotlib
!pip install accelerate
!pip install -q datasets

In [ ]:
#@title Inserire il token per loggarsi su Hugginface 🤗

#@markdown Serve per scaricare correttamente i pesi di Stable Diffusion ed eventualmente salvarsi il modello dopo
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

##(Opzionale) Se si vuole eseguire l'allenameno con la prior reservation eseguire il codice che segue per generare le foto della classe d'interesse

In [ ]:
CLASS="person" #@param {type:"string"}


#@markdown di default il numero di foto generate è 50, è possibile cambiarlo specificandolo qui sotto:
NUM_IMAGES = 50 #@param {type:"integer"}

!python generate_prior.py --gen_class $CLASS --num_images $NUM_IMAGES

In [ ]:
#@title Download delle foto generate
import shutil
from google.colab import files

# Percorso della cartella da comprimere
cartella_da_scaricare = '/content/MuDI/dataset/prior/' + CLASS

# Nome del file zip che sarà creato
nome_zip = '/content/MuDI/dataset/prior/' + CLASS + '.zip'

# Crea un file .zip dalla cartella
shutil.make_archive(base_name=nome_zip.replace('.zip', ''), format='zip', root_dir=cartella_da_scaricare)

# Scarica il file zip sul computer
files.download(nome_zip)


## Inizializzazione delle librerie per la generazione delle maschere dei riferimenti

In [ ]:
#@title import delle librerie
import torch
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import io
import re
import os

# ✅ Crea la cartella se non esiste
os.makedirs("output_masks", exist_ok=True)


In [ ]:
#@title Caricamento del modello preaddestrato SegFormer
model_name = "nvidia/segformer-b0-finetuned-ade-512-512"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Impostazione in modalità eval
model.eval()

In [ ]:
#@title Funzione per segmentare e fare le maschere delle foto

def show_segmentation_and_save_mask(image, seg_map, filename, label_ids=None, title_suffix=""):
    # Resize segmentation map
    seg_map_resized = Image.fromarray(seg_map.astype(np.uint8)).resize(image.size, resample=Image.NEAREST)
    seg_map_resized = np.array(seg_map_resized)

    # Crea la maschera binaria
    if label_ids is not None:
        mask = np.isin(seg_map_resized, label_ids)
    else:
        mask = seg_map_resized != 0  # qualunque classe diversa da sfondo

    binary_mask = (mask.astype(np.uint8)) * 255  # 0 e 255

    # ✅ Salva la maschera
    mask_image = Image.fromarray(binary_mask)
    mask_path = f"output_masks/mask_{filename}.jpg"
    mask_image.save(mask_path)

    # Mostra per confronto
    masked_image = np.array(image).copy()
    masked_image[~mask] = 0

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Immagine originale {title_suffix}")

    plt.subplot(1, 2, 2)
    plt.imshow(masked_image)
    plt.title(f"Segmentazione {title_suffix}")
    plt.axis("off")
    plt.show()

# Funzione di segmentazione
def segment_image(img_pil):
    inputs = feature_extractor(images=img_pil, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.cpu()
    pred_seg = logits.argmax(dim=1)[0].numpy()
    return pred_seg


#Upload foto


In [ ]:
#@markdown Eventualmente se si vuole segmentare altro oltre a facce e persone bisogna specificare le eticchette d'interesse

# Etichette di interesse
labels_of_interest = [12, 13]  # person, face (approx ADE20K)

# Upload dei file A
print("🔼 Carica le immagini del gruppo A (nominate a01.jpg, a02.jpg, ...)")
uploaded_a = files.upload()

# Upload dei file B
print("🔼 Ora carica le immagini del gruppo B (nominate b01.jpg, b02.jpg, ...)")
uploaded_b = files.upload()


🔼 Carica le immagini del gruppo A (nominate a01.jpg, a02.jpg, ...)


🔼 Ora carica le immagini del gruppo B (nominate b01.jpg, b02.jpg, ...)


In [ ]:
#@title Funzione per ordinare i file per nome numerico
def sort_keys(file_dict, prefix):
    return sorted(
        [k for k in file_dict if k.lower().startswith(prefix)],
        key=lambda x: int(re.findall(r'\d+', x)[0])
    )

#Segmentazione e maschere

In [ ]:
#@title Segmentazione immagini A
print("🔍 Segmentazione immagini gruppo A")
for filename in sort_keys(uploaded_a, 'a'):
    img = Image.open(io.BytesIO(uploaded_a[filename])).convert("RGB")
    seg_map = segment_image(img)
    show_segmentation_and_save_mask(img, seg_map, filename.split('.')[0], label_ids=labels_of_interest, title_suffix=f"(A - {filename})")

In [ ]:
#@title Segmentazione immagini B
print("🔍 Segmentazione immagini gruppo B")
for filename in sort_keys(uploaded_b, 'b'):
    img = Image.open(io.BytesIO(uploaded_b[filename])).convert("RGB")
    seg_map = segment_image(img)
    show_segmentation_and_save_mask(img, seg_map, filename.split('.')[0], label_ids=labels_of_interest, title_suffix=f"(A - {filename})")

In [ ]:
#@title (Opzionale) Segmentazione immagini Classe

import os
from PIL import Image

print("🔍 Segmentazione immagini immagini per la classe", CLASS)

# Percorso della cartella contenente le immagini
path_class = '/content/MuDI/dataset/prior/' + CLASS

# Ordina i file alfabeticamente (opzionale)
for filename in sorted(os.listdir(path_class)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):  # filtra immagini
        filepath = os.path.join(path_class, filename)
        img = Image.open(filepath).convert("RGB")
        seg_map = segment_image(img)
        show_segmentation_and_save_mask(
            img,
            seg_map,
            filename.split('.')[0],
            label_ids=labels_of_interest,
            title_suffix=f"(A - {filename})"
        )

#Download

In [ ]:
#@title Download della cartella contenente le maschere

import shutil
shutil.make_archive("masks_output", 'zip', "output_masks")
from google.colab import files
files.download("masks_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>